A test run for AQ module (not trained)

In [1]:
import torch
from transformers import T5Tokenizer
from models import EmbeddingLayer, PrimalDualEncoder, QuestionAnsweringOutputLayer

/common/home/sks272/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-small', model_max_length = 512)
pretrained_t5_name = 't5-small'
d_model = 512 # for t5-small

In [3]:
# Example passage, answer, and question
passage = "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France."
answer = "Eiffel Tower"
question = "What is the famous iron lattice tower located in Paris?"

In [4]:
# Tokenize the passage, answer, and question
passage_tokens = tokenizer.tokenize(passage)
question_tokens = tokenizer.tokenize(question)

In [5]:
# Create input_ids for AQ
input_ids = tokenizer.convert_tokens_to_ids(passage_tokens + question_tokens)
input_ids = torch.tensor([input_ids])  # Add batch dimension

# Create task IDs tensor (0 for question generation, 1 for question answering, 2 for KD)
task_id = 1  # Question answering
task_ids = torch.tensor([[task_id] * len(input_ids[0])])

# Create segment IDs tensor (0 for passage, 2 for question)
segment_ids = torch.tensor([[0] * len(passage_tokens) + [2] * len(question_tokens)])

In [6]:
embedding_layer = EmbeddingLayer(pretrained_t5_name, d_model)
embeddings = embedding_layer(input_ids, task_ids, segment_ids)

In [8]:
def create_attention_mask(input_ids):
    return (input_ids != tokenizer.pad_token_id)

attention_mask = create_attention_mask(input_ids)

In [9]:
# Forward pass through PrimalDualEncoder
primal_dual_encoder = PrimalDualEncoder(pretrained_t5_name)
encoder_outputs = primal_dual_encoder(embeddings, attention_mask)

In [10]:
# Instantiate the QuestionAnsweringOutputLayer
qa_output_layer = QuestionAnsweringOutputLayer(d_model)
start_scores, end_scores, start_probs, end_probs = qa_output_layer(encoder_outputs, attention_mask)

In [21]:
# Find the best start and end indices
best_start_idx = torch.argmax(start_probs, dim=-1).item()
best_end_idx = torch.argmax(end_probs, dim=-1).item()

best_start_idx, best_end_idx

(31, 38)

In [22]:
# Extract the answer span
answer_span_tokens = (passage_tokens + question_tokens)[best_start_idx:best_end_idx+1]
answer_span = tokenizer.convert_tokens_to_string(answer_span_tokens)

In [23]:
print(f"Answer span: {answer_span}")

Answer span: lattice tower located in Paris
